In [1]:
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import numpy as np
from sklearn.externals import joblib


# Select features
MDF = pd.read_csv('DataWrangling/ML.csv', index_col=0, na_values=[' ']).fillna(0)
MDF.dtypes
MDF['Gender'] = MDF['Gender'].replace({'F': 0, 'M': 1})
MDF.head()

## Convert 'Yes' to 1
cols = ['Cough', 'Fever', 'Active.Breathing.Shortness', 'Weight.Loss', 'Haemoptysis', 'TB.Medication.History']
MDF[cols] = MDF[cols].replace({'Yes':1, 'No': 0})

MDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23566 entries, 201140501469-2 to 203161000067-4
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      23566 non-null  int64  
 1   Age                         23566 non-null  int64  
 2   Cough                       23566 non-null  int64  
 3   Fever                       23566 non-null  int64  
 4   Active.Breathing.Shortness  23566 non-null  int64  
 5   Weight.Loss                 23566 non-null  int64  
 6   Haemoptysis                 23566 non-null  int64  
 7   TB.Medication.History       23566 non-null  int64  
 8   qXRv2                       23566 non-null  float64
 9   CAD4TB6                     23566 non-null  int64  
 10  JF1                         23566 non-null  float64
 11  IF2                         23566 non-null  float64
 12  Xpert2Outcome_num           23566 non-null  int64  
dtypes: float64(3),

C:\Users\zhizh\.conda\envs\RecSys\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Balance the MDF by up-sampling Minority Class  

In [2]:
MDF.Xpert2Outcome_num.value_counts()

0    19933
1     3633
Name: Xpert2Outcome_num, dtype: int64

In [3]:
from sklearn.utils import resample

# Separate majority and minority classes
MDF_majority = MDF[MDF.Xpert2Outcome_num==0]
MDF_minority = MDF[MDF.Xpert2Outcome_num==1]
 
# Upsample minority class
MDF_minority_upsampled = resample(MDF_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= 19933)    # to match majority class
#                                  random_state=321)    # reproducible results
 
# Combine majority class with upsampled minority class
MDF_upsampled = pd.concat([MDF_majority, MDF_minority_upsampled])
 
# Display new class counts
MDF_upsampled.Xpert2Outcome_num.value_counts()


1    19933
0    19933
Name: Xpert2Outcome_num, dtype: int64

### Test & train split

In [4]:
## Balanced MDF
array = MDF_upsampled.values
# array = MDF.values

# split into input (X) and output (Y) variables
X = array[:,[0,1,2,3,4,5,6, 7]] # take the first few columns as x to predict 'class'
Y = array[:,11] # separate 'class' variable as the predictor

# encode class values as integers
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)


## Original MDF
dfi = MDF.values
# split into input (X) and output (Y) variables
Xi = dfi[:,[0,1,2,4,5,6,7,8]].astype(float)
Yi = dfi[:,3]


# ML Models

In [ ]:
print(MDF_upsampled.describe())

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X, encoded_Y, test_size=0.20)
# We set the random seed via the random_state argument to a fixed number to ensure that each algorithm is evaluated on the same splits of the training MDF.


# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('RFC100', RandomForestClassifier(n_estimators=100, random_state=42)))
models.append(('RFC10_42', RandomForestClassifier(n_estimators=10, random_state=42)))

# models.append(('SVM', SVC(gamma='auto')))

# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = StratifiedKFold(n_splits=10, random_state=1)
	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='roc_auc')
	results.append(cv_results)
	names.append(name)
	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    
# Compare Algorithms
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()


# Select the model

In [ ]:
from sklearn.metrics import plot_roc_curve
X_train, X_validation, Y_train, Y_validation = train_test_split(X, encoded_Y, test_size=0.20)

rfc = RandomForestClassifier(n_estimators=200, random_state=42)
rfc.fit(X_train, Y_train)
ax = plt.gca()
rfc_disp = plot_roc_curve(rfc, X_train, Y_train, ax=ax, alpha=0.8)
plt.show()

# print(rfc.feature_importances_)
# print(rfc.predict(X_validation))

# joblib.dump(rfc, 'Results/trained_RandomForest_model.pkl')

### Use the balanced MDF to validate

In [ ]:
ax = plt.gca()
rfc_disp = plot_roc_curve(rfc, X_validation, Y_validation, ax=ax, alpha=0.8)
plt.show()

### Recover the original MDF

In [ ]:
allPred = rfc.predict_proba(Xi)[:,1]

# print(allPred[:,0])
MDF['IndividualRiskScore'] = pd.DataFrame(allPred, columns=['pred'])

MDF.to_csv(r'Results\pred_MDF.csv', index = False, header = True)

### Visualize the performance of  a Logistic Regression Binary Classifier

In [ ]:
from VisualAssess import evalBinaryClassifier
F1 = evalBinaryClassifier(rfc, X_validation, Y_validation)

In [ ]:
    cm = confusion_matrix(Y_validation, X_validation[:, 2])
    print(cm)

# Visualize a Decision Tree from a Random Forest 

In [ ]:
# NOT COMPLETE
from sklearn.ensemble import RandomForestClassifier

# Extract single tree
estimator = rfc.estimators_[5]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = iris.feature_names,
                class_names = iris.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')